<a href="https://colab.research.google.com/github/Bilal-Ahmad-5/Fraud-Detection-Agent/blob/main/Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet --upgrade langgraph langchain_experimental langchain langchain_huggingface langchain_community gradio Chroma faiss-cpu langchain-chroma langchain-google-genai langchain-groq

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━

In [2]:
from google.colab import userdata
import os

os.environ["TAVILY_API_KEY"] = userdata.get('tavily_api_key')
os.environ["GROQ_API_KEY"] = userdata.get('groq_api_key')
os.environ["GOOGLE_API_KEY"] = userdata.get('Gemini_Api_Key')
os.environ["LANGCHAIN_API_KEY"] = userdata.get('langchai_api_key')
os.environ["LANGCHAIN_PROJECT"] = "Fraud-Detection-Agent"
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [ ]:
import kagglehub
import pandas as pd
import os

# Download latest version
path = kagglehub.dataset_download("sriharshaeedala/financial-fraud-detection-dataset")

print("Path to dataset files:", path)


# Step 2: Look at the downloaded files
files = os.listdir(path)
print("Files in dataset directory:", files)

csv_filename = "Synthetic_Financial_datasets_log.csv"
csv_path = os.path.join(path, csv_filename)

# Load the dataset into a pandas DataFrame
df = pd.read_csv(csv_path)
print("Data loaded. Number of records:", len(df))

100%|██████████| 178M/178M [00:01<00:00, 104MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/sriharshaeedala/financial-fraud-detection-dataset/versions/1
Files in dataset directory: ['Synthetic_Financial_datasets_log.csv']


In [ ]:
import os
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_chroma import Chroma
from langchain.docstore.document import Document
from langchain.chains import RetrievalQA
from langchain.agents import initialize_agent, Tool, AgentType
import gradio as gr
import re
import numpy as np
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.tools import tool
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader , WebBaseLoader , CSVLoader



def csv_loader(file):
    loader = CSVLoader(file)
    data = loader.load()
    return data

# Data Splitter
chunk_size = 500
chunk_overlap = 50

def text_splitter(data):
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=chunk_size,
      chunk_overlap=chunk_overlap
    )
  split_text = text_splitter.split_documents(data)
  return split_text

## Vector db
def vector_database(chunks):
    embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")

    vector_store = Chroma(
        collection_name="example_collection",
        embedding_function=embedding_model,
    )
    # Add documents to vector store
    vector_store.add_documents(chunks)
    return vector_store

## Retriever
def retriever(file):
    splits = csv_loader(file)
    chunks = text_splitter(splits)
    vectordb = vector_database(chunks)
    retriever = vectordb.as_retriever()
    return retriever


In [ ]:
# Define specialized tool functions
# -------------------------------

def fraud_retrieve(query: str) -> str:
    """
    Tool that retrieves fraud detection rules and best practices
    based on the input query using a RAG approach.
    """
    Retriever = retriever(csv_path)
    response = Retriever.run(query)
    return response

def analyze_transaction(transaction_details: str) -> str:
    """
    Tool that analyzes provided transaction details for potential fraud risk.
    It generates a detailed analysis along with a risk score (out of 10),
    highlighting red flags and suggesting further steps.
    """
    prompt = f"""
You are a fraud detection expert. Evaluate the following transaction details and provide a detailed analysis regarding its potential fraud risk. Identify any red flags and suggest further verification steps. Also, score the risk on a scale from 1 (low risk) to 10 (high risk).

Transaction Details:
{transaction_details}

Consider common fraud detection rules such as:
- Unusually high amounts
- Suspicious geographical locations
- Multiple rapid transactions
- Sudden deviations from historical patterns

Provide a clear, step-by-step explanation of your reasoning.
    """
    # Use an LLM to generate the analysis.
    analysis = ChatGroq(temperature=0.3)(prompt)
    return analysis

# -------------------------------
# Assemble the Fraud Detection Agent Team
# -------------------------------

# Define the tools that the agent can use.
tools = [
    Tool(
        name="Fraud Knowledge Retrieval",
        func=fraud_retrieve,
        description=("Retrieves relevant fraud detection rules and insights from the knowledge base "
                     "based on your query. Use this tool when you need background or strategy information.")
    ),
    Tool(
        name="Transaction Analysis",
        func=analyze_transaction,
        description=("Analyzes specific transaction details for potential fraud risk. "
                     "Use this tool when you want a detailed risk assessment of a given transaction scenario.")
    )
]

# Initialize the LangChain agent with the above tools.
llm = ChatGroq(temperature=0.3)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# -------------------------------
# Expose the Agent via a Gradio Interface
# -------------------------------

def run_agent(user_input: str) -> str:
    """
    This function allows the agent to process the user input.
    Depending on the query it may decide to retrieve fraud rules, analyze a transaction,
    or even call both tools in a collaborative fashion.
    """
    result = agent.run(user_input)
    return result

# Create Gradio interface
rag_application = gr.Interface(
    fn=run_agent,
    allow_flagging="never",
    inputs=[
        gr.Textbox(label="Input Query", lines=2, placeholder="Type your question here...")
    ],
    outputs=gr.Textbox(label="Output"),
    title="Fraud Detection"
)

if __name__ == "__main__":
    # Launch the Gradio web interface.
    rag_application.launch()
